In [1]:
from nav2_simple_commander.robot_navigator import BasicNavigator
import rclpy

rclpy.init()
nav = BasicNavigator()

In [2]:
nav.waitUntilNav2Active()

[INFO] [1719217008.732222967] [basic_navigator]: Nav2 is ready for use!


In [3]:
from geometry_msgs.msg import PoseStamped

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.position.x = 1.77
goal_pose.pose.position.y = 0.6
goal_pose.pose.position.z = 0.0
goal_pose.pose.orientation.x = 0.0
goal_pose.pose.orientation.y = 0.0
goal_pose.pose.orientation.z = 0.0
goal_pose.pose.orientation.w = 1.0

In [4]:
nav.goToPose(goal_pose)

[INFO] [1719217008.765890377] [basic_navigator]: Navigating to goal: 1.77 0.6...


True

In [5]:
from rclpy.duration import Duration

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')

    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
        nav.cancelTack()

Distance remaining1.77meters.
Distance remaining1.67meters.
Distance remaining1.52meters.
Distance remaining1.42meters.
Distance remaining1.32meters.
Distance remaining1.16meters.
Distance remaining1.05meters.
Distance remaining0.89meters.
Distance remaining0.80meters.
Distance remaining0.70meters.
Distance remaining0.55meters.
Distance remaining0.44meters.
Distance remaining0.39meters.
Distance remaining0.27meters.
Distance remaining0.29meters.
Distance remaining0.29meters.
Distance remaining0.29meters.
Distance remaining0.29meters.


In [6]:
from nav2_simple_commander.robot_navigator import TaskResult

result = nav.getResult()
if result == TaskResult.SUCCEEDED:
    print('Goal succeeded!')
elif result == TaskResult.CANCELED:
    print('Goal was canceled!')
elif result == TaskResult.FAILED:
    print('Goal failed!')

Goal succeeded!


In [7]:
import rclpy
from geometry_msgs.msg import PoseWithCovarianceStamped

rclpy.shutdown() 
rclpy.init()
pose_current = PoseWithCovarianceStamped()
pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [8]:
pose_current.pose.pose.position

geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0)

In [9]:
pose_current.pose.pose.orientation

geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

In [10]:
# pose_node = rclpy.create_node('sub_pose')

# def callback(data):
#     print("===")
#     print(data.pose.pose.position)
#     print(data.pose.pose.orientation)

pose_node = rclpy.create_node('sub_pose')

def callback(data):
    global pose_current
    print("===")
    print(data.pose.pose.position)
    print(data.pose.pose.orientation)

In [11]:
pose_node.create_subscription(PoseWithCovarianceStamped, '/amcl_pose', callback, 10)

In [12]:
rclpy.spin_once(pose_node)

===
geometry_msgs.msg.Point(x=1.6656818861134817, y=0.4553336545330968, z=0.0)
geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.044843918821477556, w=0.998994005459859)


In [13]:
from tf_transformations import euler_from_quaternion

In [14]:
pose_current

geometry_msgs.msg.PoseWithCovarianceStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)), covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])))

In [15]:
pose_current.pose.pose.orientation

geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)

In [16]:
q = [0, 0, 0, 0]
q[0] = pose_current.pose.pose.orientation.x
q[1] = pose_current.pose.pose.orientation.y
q[2] = pose_current.pose.pose.orientation.z
q[3] = pose_current.pose.pose.orientation.w

euler_from_quaternion(q)

(0.0, -0.0, 0.0)

In [17]:
import numpy as np

def convert_degree(input):
    return np.array(input) * 180. / np.pi

convert_degree(euler_from_quaternion(q))

array([ 0., -0.,  0.])

In [18]:
from tf_transformations import quaternion_from_euler

to_radian = np.pi / 180.
tmp = [0, 0, 0]
print(np.array(tmp)*to_radian)

quaternion_from_euler(tmp[0], tmp[1], tmp[2])

[0. 0. 0.]


(0.0, 0.0, 0.0, 1.0)

In [19]:
from nav2_simple_commander.robot_navigator import BasicNavigator
from geometry_msgs.msg import PoseStamped
nav = BasicNavigator()

target = quaternion_from_euler(tmp[0], tmp[1], tmp[2])
goal_pose = PoseStamped()

goal_pose.header.frame_id = 'map'
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.orientation.x = target[0]
goal_pose.pose.orientation.y = target[1]
goal_pose.pose.orientation.z = target[2]
goal_pose.pose.orientation.w = target[3]

In [20]:
from rclpy.duration import Duration

nav.goToPose(goal_pose)

i = 0
while not nav.isTaskComplete():
    i = i + 1
    feedback = nav.getFeedback()
    if feedback and i % 5 == 0:
        print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + ' meters.')
        
        if Duration.from_msg(feedback.navigation_time) > Duration(seconds=10.0):
            nav.cancelTask()

[INFO] [1719217027.810348756] [basic_navigator]: Navigating to goal: 0.0 0.0...


Distance remaining: 2.02 meters.
Distance remaining: 2.02 meters.
Distance remaining: 2.02 meters.
Distance remaining: 2.02 meters.
Distance remaining: 1.95 meters.
Distance remaining: 1.90 meters.
Distance remaining: 1.75 meters.
Distance remaining: 1.65 meters.
Distance remaining: 1.50 meters.
Distance remaining: 1.40 meters.
Distance remaining: 1.30 meters.
Distance remaining: 1.15 meters.
Distance remaining: 1.05 meters.
Distance remaining: 0.90 meters.
Distance remaining: 0.80 meters.
Distance remaining: 0.65 meters.
Distance remaining: 0.55 meters.
Distance remaining: 0.40 meters.
Distance remaining: 0.30 meters.
Distance remaining: 0.20 meters.
Distance remaining: 0.20 meters.


[INFO] [1719217038.380071491] [basic_navigator]: Canceling current task.
